In [140]:
from __future__ import division
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import os
import matplotlib
import numpy as np
import random
from numpy import linalg as LA
from PIL import Image
import sys

In [141]:
# '''This is the command to read in the GIF. You need Videocapture which makes a cv2 Video Object.
# Add your GIF file name in the parameter of this function.
# '''
# #gif_kiss = cv2.VideoCapture('test.gif')


# def convert_gif_to_frames(gif):

#     # Initialize the frame number and create empty frame list
#     frame_num = 0
#     frame_list = []

#     # Loop until there are frames left
#     while True:
#         try:
#             # Try to read a frame. Okay is a BOOL if there are frames or not
#             okay, frame = gif.read()
#             # Append to empty frame list
#             frame_list.append(frame)
#             # Break if there are no other frames to read
#             if not okay:
#                 break
#             # Increment value of the frame number by 1
#             frame_num += 1
#         except KeyboardInterrupt:  # press ^C to quit
#             break

#     return frame_list


# def output_frames_as_pics(frame_list):

#     # Reduce the list of frames by half to make the list more managable
#     frame_list_reduce = frame_list[0::2]
#     # Get the path of the current working directory
#     path = os.getcwd()
#     # Set then name of your folder
#     '''Replace this name with what you want your folder name to be'''
#     folder_name = 'Picturebook_Pics_Kiss'
#     # If the folder does not exist, then make it
#     if not os.path.exists(path + '/' + folder_name):
#         os.makedirs(path + '/' + folder_name)

#     for frames_idx in range(len(frame_list_reduce)):
#         cv2.imwrite(os.path.join(path + '/' + folder_name, str(frames_idx+1) + '.png'), frame_list_reduce[frames_idx])

# pass

In [142]:
# cap = cv2.VideoCapture('test1.gif')
# a = convert_gif_to_frames(cap)
# cv2.imshow('Original image',a[0])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [143]:
# a = convert_gif_to_frames(cap)

In [144]:
# img = Image.open('test.gif') # image extension *.png,*.jpg
# new_width  = 50
# new_height = 50
# img = img.resize((new_width, new_height), Image.ANTIALIAS)
# img.save('resize_images/test1.gif') # format may what u want ,*.png,*jpg,*.gif

In [145]:
# cap = cv2.imread('Aberdeen/lisa1.jpg')
# gray = cv2.cvtColor(cap, cv2.COLOR_BGR2GRAY)
# cv2.imshow('Original image',cap)
# cv2.imshow('Gray image', gray)
 
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [146]:
# len(a[0][0])

In [147]:
# len(cap[0][0])

In [148]:
# len(gray[0])

In [149]:
# plt.imshow(gray,cmap='gray')
# plt.show()

In [150]:
# images = []

In [151]:
# images

In [152]:
# new_width  = 50
# new_height = 50
# for root, dirs, files in os.walk('./face_dataset'):
#     k = 0
#     for name in files:
#         #print(name)
#         cap = cv2.VideoCapture('./face_dataset/'+name)
#         img = Image.open('./face_dataset/'+name) # image extension *.png,*.jpg
#         img = img.resize((new_width, new_height), Image.ANTIALIAS)
#         img.save('resize_images/'+name) # format may what u want ,*.png,*jpg,*.gif

In [153]:
# new_width  = 50
# new_height = 50
# for root, dirs, files in os.walk('./Aberdeen'):
#     k = 0
#     for name in files:
#         #print(name)
#         #cap = cv2.VideoCapture('./face_dataset/'+name))
#         img = Image.open('./Aberdeen/'+name).convert('LA') # image extension *.png,*.jpg
#         img = img.resize((new_width, new_height), Image.ANTIALIAS)
#         img.save('aberdeen_resize_images/'+name) # format may what u want ,*.png,*jpg,*.gif

In [154]:
img_name = []
d = {}
key = ''
for root, dirs, files in os.walk('./aberdeen_resize_images'):
    if not (root == './aberdeen_resize_images'):
        key = root.split('\\')[1]
        d[key] = []
    for name in files:
        img_name.append(name)
#         d[key].append(name)
        cap = cv2.imread('./aberdeen_resize_images/'+key+'/'+name)
#         a = convert_gif_to_frames(cap)
#         print(a[0][:])
        d[key].append(cap)
#         plt.imshow(a[0])
#         plt.show()

In [155]:
# d

In [156]:
# len(images)

In [157]:
def l1pca_SBF_rank1_simplified(dfX,L):
    N = len(dfX.T) #Number of data points
    max_iter = 25 #Number of iteration    
    delta = [0] * N #array to store quadratic value
    obj_val = 0 #initialize the objective function's value ||X*b||_2^2
    
    for i in range(0,L): #loop through initializations
#         b = []
#         for j in range(0,N): #random initalization of the b vector
#             temp = 0
#             while temp == 0:
#                 temp = random.randint(-1,1)
#             b.append(temp)
#         b = np.array(b)
        b = np.random.choice([0, 1], size=(N,), p=[1./3, 2./3])* 2 - 1;
        dfb = pd.DataFrame(b)
        print('Initializtion : ',i)
        for j in range(0,max_iter): #loop of iterations for bit flipping
            for k in range(0,N): #loop over N bits in b
#                 print('Initialization: ',i,'Iteration: ',j,'Sample: ',k)
                dfbk = dfb.copy()
                dfXk = dfX.copy()
                dfbk.drop([k])
                dfXk.drop([k],axis=1)
#                 bk = np.delete(b,k,0) #remove the i-th bit
#                 Xk = np.delete(X,k,1) #remove the i-th data sample
                delta[k] = int(-4 * (dfX.loc[:,0]*int(dfb.iloc[0])).dot(dfX.dot(dfb))) #calculate the "increase of the objective function" after flipping the i-th bit 
            ID = sorted(range(len(delta)), key=lambda g: delta[g],reverse = True) #sort delta and find the bit that leads to the highest increase
            #delta.sort(reverse = True)
            val = np.sort(delta)[::-1]
            
            if val[0] > 0: #if the highest "increase" is positive
                b[ID[0]] = -b[ID[0]] #then flip the corresponding bit in b
            else:
#                 print(j)
                break #otherwise, SBF terminates
        tmp = LA.norm(dfX.dot(dfb)) #calculate the  objective function's value
        if tmp > obj_val: #if larger than the old obj function value
            obj_val = tmp
            bopt = b #then update the obj function value, and the optimal b
            l_best = i
    Qprop = dfX.dot(dfb)/LA.norm(dfX.dot(dfb)) #PCA X*bopt/norm(X*bopt)
    Bprop = bopt
    return Qprop,Bprop,l_best

In [158]:
# images = np.array(images)

In [159]:
# images[0][0][0][0]

In [160]:
# len(d['lisa'][0][0])

In [161]:
ddtrain = {}
ddtest = {}

In [162]:

for i in d:
    item = 3
    ddtrain[i] = []
    ddtest[i] = []
    for j in d[i]:
        X = []
        for k in j:
            for l in k:
                X.append(k[0][0])
        if item > 0:
            ddtest[i].append(X[:])
        else:
            ddtrain[i].append(X[:])
        item -= 1

In [163]:
for i in ddtrain:
    print(i,len(ddtest[i]))

lisa 3
louise 3
marie 3
michael 3
nick 3
pat 3
stephen 3
trevor 3


In [164]:
ddmean = pd.DataFrame()
for i in ddtrain:
    ddtrain[i] = pd.DataFrame(ddtrain[i])
    ddtrain[i] = ddtrain[i].T
    ddmean[i+'_'+'mean'] = ddtrain[i].sum(axis=1)/14

In [165]:
ddmean

,lisa_mean,louise_mean,marie_mean,michael_mean,nick_mean,pat_mean,stephen_mean,trevor_mean
0,268.785714,267.785714,268.642857,269.571429,265.428571,268.571429,269.142857,268.071429
1,268.785714,267.785714,268.642857,269.571429,265.428571,268.571429,269.142857,268.071429
2,268.785714,267.785714,268.642857,269.571429,265.428571,268.571429,269.142857,268.071429
3,268.785714,267.785714,268.642857,269.571429,265.428571,268.571429,269.142857,268.071429
4,268.785714,267.785714,268.642857,269.571429,265.428571,268.571429,269.142857,268.071429
5,268.785714,267.785714,268.642857,269.571429,265.428571,268.571429,269.142857,268.071429
6,268.785714,267.785714,268.642857,269.571429,265.428571,268.571429,269.142857,268.071429
7,268.785714,267.785714,268.642857,269.571429,265.428571,268.571429,269.142857,268.071429
8,268.785714,267.785714,268.642857,269.571429,265.428571,268.571429,269.142857,268.071429
9,268.785714,267.785714,268.642857,269.571429,265.428571,268.571429,269.142857,268.071429


In [166]:
for i in ddtrain:
    print('Subtracting mean for :',i)
    for j in list(ddtrain[i]):
        if j == 'mean':
            continue
        ddtrain[i][j] = ddtrain[i][j] - ddmean[i+'_'+'mean']

Subtracting mean for : lisa
Subtracting mean for : louise
Subtracting mean for : marie
Subtracting mean for : michael
Subtracting mean for : nick
Subtracting mean for : pat
Subtracting mean for : stephen
Subtracting mean for : trevor


In [167]:
for i in ddtest:
    ddtest[i] = pd.DataFrame(ddtest[i])
    ddtest[i] = ddtest[i].T

In [168]:
dddtest = pd.DataFrame()
for i in ddtest:
    for j in list(ddtest[i]):
        dddtest[i+'_'+str(j)] = ddtest[i][j]

In [169]:
ddtrain['lisa']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-14.785714,-23.785714,-30.785714,-14.785714,-14.785714,-13.785714,-13.785714,-13.785714,-14.785714,-13.785714,-13.785714,-14.785714,-13.785714,-29.785714,-27.785714
1,-14.785714,-23.785714,-30.785714,-14.785714,-14.785714,-13.785714,-13.785714,-13.785714,-14.785714,-13.785714,-13.785714,-14.785714,-13.785714,-29.785714,-27.785714
2,-14.785714,-23.785714,-30.785714,-14.785714,-14.785714,-13.785714,-13.785714,-13.785714,-14.785714,-13.785714,-13.785714,-14.785714,-13.785714,-29.785714,-27.785714
3,-14.785714,-23.785714,-30.785714,-14.785714,-14.785714,-13.785714,-13.785714,-13.785714,-14.785714,-13.785714,-13.785714,-14.785714,-13.785714,-29.785714,-27.785714
4,-14.785714,-23.785714,-30.785714,-14.785714,-14.785714,-13.785714,-13.785714,-13.785714,-14.785714,-13.785714,-13.785714,-14.785714,-13.785714,-29.785714,-27.785714
5,-14.785714,-23.785714,-30.785714,-14.785714,-14.785714,-13.785714,-13.785714,-13.785714,-14.785714,-13.785714,-13.785714,-14.785714,-13.785714,-29.785714,-27.785714
6,-14.785714,-23.785714,-30.785714,-14.785714,-14.785714,-13.785714,-13.785714,-13.785714,-14.785714,-13.785714,-13.785714,-14.785714,-13.785714,-29.785714,-27.785714
7,-14.785714,-23.785714,-30.785714,-14.785714,-14.785714,-13.785714,-13.785714,-13.785714,-14.785714,-13.785714,-13.785714,-14.785714,-13.785714,-29.785714,-27.785714
8,-14.785714,-23.785714,-30.785714,-14.785714,-14.785714,-13.785714,-13.785714,-13.785714,-14.785714,-13.785714,-13.785714,-14.785714,-13.785714,-29.785714,-27.785714
9,-14.785714,-23.785714,-30.785714,-14.785714,-14.785714,-13.785714,-13.785714,-13.785714,-14.785714,-13.785714,-13.785714,-14.785714,-13.785714,-29.785714,-27.785714


In [170]:
dddtest[['lisa_0']]

,lisa_0
0,255
1,255
2,255
3,255
4,255
5,255
6,255
7,255
8,255
9,255


In [171]:
# df = pd.DataFrame.from_dict(dd, orient='index')

In [172]:
# item = 0
# X = []
# for i in images:
#     tmp = []
#     for j in i:
#         for k in j:
#             tmp.append(k[0])
#     X.append(tmp)
#     item += 1

In [173]:
# len(X[90])

In [174]:
# len(images[0][0])

In [175]:
# m = 99999999999999
# for i in X:
#     m = min(len(i),m)

In [176]:
# m

In [177]:
# for i in range(0,len(X)):
#     X[i] = X[i][0:m]

In [178]:
# X = np.array(X)

In [179]:
# random.randrange(-1, 1, 2)

In [180]:
# len(X.T)

In [181]:
# b = [-1]*100

In [182]:
# dfX = pd.DataFrame(X)

In [183]:
# dfX

In [184]:
# dfb = pd.DataFrame(b)

In [185]:
#dfX.T.dot(dfb)/LA.norm(dfX.T.dot(dfb))

In [186]:
# (dfX.T.loc[:,0]*int(dfb.iloc[0])).dot(dfX.T.dot(dfb))

In [187]:
# df = pd.DataFrame([[0, 1, -2, -1], [1, 1, 1, 1]])
# other = pd.DataFrame([[0, 1], [1, 2], [-1, -1], [2, 0]])
# df.dot(other)

In [188]:
# df = pd.DataFrame

In [189]:
# df.iloc[0] = 0

In [190]:
# other

In [191]:
# dfX = pd.DataFrame(ddtrain['lisa'])

In [192]:
# len(dfX.T)

In [193]:
# Qprop,Bprop,l_best = l1pca_SBF_rank1_simplified(dfX.T,10)

In [194]:
# Qprop

In [195]:
K = 20

In [196]:
dfrL1 = pd.DataFrame()
for k in range(1,K+1):
    print('Principal Component: ',k)
    for i in ddtrain:
        Qprop,Bprop,l_best = l1pca_SBF_rank1_simplified(ddtrain[i],10)
        print(i,l_best)
        dfrL1[str(k)+'_'+str(i)] = Qprop[0].values
    for i in ddtrain:
        ddtrain[i] = ddtrain[i] - dfrL1[[str(k)+'_'+str(i)]].dot(dfrL1[[str(k)+'_'+str(i)]].T.dot(ddtrain[i]))

Principal Component:  1
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
lisa 9
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
louise 5
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
marie 9
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
michael 8
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializti

Initializtion :  9
lisa 3
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
louise 2
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
marie 7
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
michael 1
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
nick 9
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializ

Initializtion :  7
Initializtion :  8
Initializtion :  9
louise 5
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
marie 3
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
michael 4
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
nick 2
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
pat 9
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializt

Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
marie 8
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
michael 5
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
nick 4
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
pat 9
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializtion :  4
Initializtion :  5
Initializtion :  6
Initializtion :  7
Initializtion :  8
Initializtion :  9
stephen 9
Initializtion :  0
Initializtion :  1
Initializtion :  2
Initializtion :  3
Initializ

In [197]:
#ddtrain['lisa']

In [198]:
#ddtrain['lisa'] - dfrL1[['1_lisa']].dot(dfrL1[['1_lisa']].T.dot(ddtrain['lisa']))

In [199]:
#a = ['1_lisa','2_lisa']

In [200]:
#dfrL1[a].dot(dfrL1[a].T)

In [201]:
dfrL1

,1_lisa,1_louise,1_marie,1_michael,1_nick,1_pat,1_stephen,1_trevor,2_lisa,2_louise,...,19_stephen,19_trevor,20_lisa,20_louise,20_marie,20_michael,20_nick,20_pat,20_stephen,20_trevor
0,-0.019255,-0.002869,-0.012304,0.013746,-0.003741,-0.017189,-0.021373,0.001546,0.005086,0.022127,...,0.006443,0.001121,0.013663,0.019232,0.005383,-0.005373,0.019668,-0.018940,0.031559,0.028991
1,-0.019255,-0.002869,-0.012304,0.013746,-0.003741,-0.017189,-0.021373,0.001546,0.005086,0.022127,...,0.006443,0.001121,0.013663,0.019232,0.005383,-0.005373,0.019668,-0.018940,0.031559,0.028991
2,-0.019255,-0.002869,-0.012304,0.013746,-0.003741,-0.017189,-0.021373,0.001546,0.005086,0.022127,...,0.006443,0.001121,0.013663,0.019232,0.005383,-0.005373,0.019668,-0.018940,0.031559,0.028991
3,-0.019255,-0.002869,-0.012304,0.013746,-0.003741,-0.017189,-0.021373,0.001546,0.005086,0.022127,...,0.006443,0.001121,0.013663,0.019232,0.005383,-0.005373,0.019668,-0.018940,0.031559,0.028991
4,-0.019255,-0.002869,-0.012304,0.013746,-0.003741,-0.017189,-0.021373,0.001546,0.005086,0.022127,...,0.006443,0.001121,0.013663,0.019232,0.005383,-0.005373,0.019668,-0.018940,0.031559,0.028991
5,-0.019255,-0.002869,-0.012304,0.013746,-0.003741,-0.017189,-0.021373,0.001546,0.005086,0.022127,...,0.006443,0.001121,0.013663,0.019232,0.005383,-0.005373,0.019668,-0.018940,0.031559,0.028991
6,-0.019255,-0.002869,-0.012304,0.013746,-0.003741,-0.017189,-0.021373,0.001546,0.005086,0.022127,...,0.006443,0.001121,0.013663,0.019232,0.005383,-0.005373,0.019668,-0.018940,0.031559,0.028991
7,-0.019255,-0.002869,-0.012304,0.013746,-0.003741,-0.017189,-0.021373,0.001546,0.005086,0.022127,...,0.006443,0.001121,0.013663,0.019232,0.005383,-0.005373,0.019668,-0.018940,0.031559,0.028991
8,-0.019255,-0.002869,-0.012304,0.013746,-0.003741,-0.017189,-0.021373,0.001546,0.005086,0.022127,...,0.006443,0.001121,0.013663,0.019232,0.005383,-0.005373,0.019668,-0.018940,0.031559,0.028991
9,-0.019255,-0.002869,-0.012304,0.013746,-0.003741,-0.017189,-0.021373,0.001546,0.005086,0.022127,...,0.006443,0.001121,0.013663,0.019232,0.005383,-0.005373,0.019668,-0.018940,0.031559,0.028991


In [202]:
ddtrain.keys()

dict_keys(['lisa', 'louise', 'marie', 'michael', 'nick', 'pat', 'stephen', 'trevor'])

In [203]:
list(ddmean)

['lisa_mean',
 'louise_mean',
 'marie_mean',
 'michael_mean',
 'nick_mean',
 'pat_mean',
 'stephen_mean',
 'trevor_mean']

In [204]:
df = dddtest['lisa_0'].copy

In [205]:
df = pd.DataFrame()
df[0] = dddtest['lisa_0'].copy()

In [206]:
-sys.maxsize

-2147483647

In [207]:
len(list(dddtest))

24

In [208]:
tmp_dddtest = dddtest[['lisa_0']]

In [209]:
error = sys.maxsize
min_error = sys.maxsize
test_label = ''
true_label = ''
labels = []
for principal_component_i in range(1,K+1):
    labels = []
    print('Principal_Component :',principal_component_i)
    for test_image_i in dddtest:
        true_label = test_image_i.split('_')[0]
        for class_c in ddtrain.keys():
            lst = []
            for i in range(1,principal_component_i+1):
                lst.append(str(i)+'_'+class_c)
            #print(lst)
            QQT = dfrL1[lst].dot(dfrL1[lst].T)
            df = pd.DataFrame()
            df[0] = dddtest[test_image_i].copy()
            error =  LA.norm((df[0] - ddmean[class_c+'_'+'mean']) - QQT.dot((df[0] - ddmean[class_c+'_'+'mean'])))
            if error < min_error:
                min_error = error
                test_label = class_c
        #print('Test Image :',test_image_i,'True Label :',true_label,'Test Label',test_label)
        l = []
        l.append(true_label)
        l.append(test_label)
        labels.append(l)
    accurate = 0
    for i in labels:
        if i[0] == i[1]:
            accurate += 1
    print('Accurate :',accurate,'Wrong :',(len(list(dddtest))-accurate))

Principal_Component : 1
Accurate : 0 Wrong : 24
Principal_Component : 2
Accurate : 3 Wrong : 21
Principal_Component : 3
Accurate : 3 Wrong : 21
Principal_Component : 4
Accurate : 3 Wrong : 21
Principal_Component : 5
Accurate : 3 Wrong : 21
Principal_Component : 6
Accurate : 3 Wrong : 21
Principal_Component : 7
Accurate : 3 Wrong : 21
Principal_Component : 8
Accurate : 3 Wrong : 21
Principal_Component : 9
Accurate : 3 Wrong : 21
Principal_Component : 10
Accurate : 3 Wrong : 21
Principal_Component : 11
Accurate : 3 Wrong : 21
Principal_Component : 12
Accurate : 3 Wrong : 21
Principal_Component : 13
Accurate : 3 Wrong : 21
Principal_Component : 14
Accurate : 4 Wrong : 20
Principal_Component : 15
Accurate : 3 Wrong : 21
Principal_Component : 16
Accurate : 3 Wrong : 21
Principal_Component : 17
Accurate : 3 Wrong : 21
Principal_Component : 18
Accurate : 3 Wrong : 21
Principal_Component : 19
Accurate : 3 Wrong : 21
Principal_Component : 20
Accurate : 3 Wrong : 21


In [210]:
accurate = 0
for i in labels:
    if i[0] == i[1]:
        accurate += 1
accurate

3

In [211]:
len(labels)

24

In [212]:
df[0]

0       255
1       255
2       255
3       255
4       255
5       255
6       255
7       255
8       255
9       255
10      255
11      255
12      255
13      255
14      255
15      255
16      255
17      255
18      255
19      255
20      255
21      255
22      255
23      255
24      255
25      255
26      255
27      255
28      255
29      255
       ... 
2470     54
2471     54
2472     54
2473     54
2474     54
2475     54
2476     54
2477     54
2478     54
2479     54
2480     54
2481     54
2482     54
2483     54
2484     54
2485     54
2486     54
2487     54
2488     54
2489     54
2490     54
2491     54
2492     54
2493     54
2494     54
2495     54
2496     54
2497     54
2498     54
2499     54
Name: 0, Length: 2500, dtype: int64

In [213]:
LA.norm((df[0] - ddmean['lisa'+'_'+'mean'])-QQT.dot((df[0] - ddmean['lisa'+'_'+'mean'])))

1197.3562659319487

In [214]:
LA.norm((df[0] - ddmean['lisa'+'_'+'mean']) - QQT.dot((df[0] - ddmean['lisa'+'_'+'mean'])))

1197.3562659319487